<a href="https://colab.research.google.com/github/shriya-tiwari/NLP-B_Tech_Project/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import numpy as np

import re
import nltk
nltk.download('all')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemm = WordNetLemmatizer()

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

In [2]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [3]:
data = pd.read_csv("/content/gdrive/My Drive/Clothing Review.csv")
print(data.head(7))
print(data.shape)

# removing null values
data = data[data['Class Name'].isnull() == False]

   Clothing ID  Age                    Title  \
0          767   33                      NaN   
1         1080   34                      NaN   
2         1077   60  Some major design flaws   
3         1049   50         My favorite buy!   
4          847   47         Flattering shirt   
5         1080   49  Not for the very petite   
6          858   39     Cagrcoal shimmer fun   

                                         Review Text  Rating  Recommended IND  \
0  Absolutely wonderful - silky and sexy and comf...       4                1   
1  Love this dress!  it's sooo pretty.  i happene...       5                1   
2  I had such high hopes for this dress and reall...       3                0   
3  I love, love, love this jumpsuit. it's fun, fl...       5                1   
4  This shirt is very flattering to all due to th...       5                1   
5  I love tracy reese dresses, but this one is no...       2                0   
6  I aded this in my basket at hte last mintue t

In [4]:
# text columns are used to as features and rating column is used for sentiments
def filter_score(rating):
	return int(rating > 3) #rating above 3 is made as 1 and below 3 as 0

features = ['Class Name', 'Title', 'Review Text']

X = data[features]
y = data['Rating']
y = y.apply(filter_score)

In [14]:
def toLower(data):
    if isinstance(data, float):
        return '<UNK>'
    else:
        return data.lower()

X['Title'] = X['Title'].apply(toLower)
X['Review Text'] = X['Review Text'].apply(toLower)

<ipython-input-14-3f0132498f92>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Title'] = X['Title'].apply(toLower)
<ipython-input-14-3f0132498f92>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Review Text'] = X['Review Text'].apply(toLower)


In [ ]:
stop_words = stopwords.words("english")
def remove_stopwords(text):
	no_stop = []
	for word in text.split(' '):
		if word not in stop_words:
			no_stop.append(word)
	return " ".join(no_stop)


X['Title'] = X['Title'].apply(remove_stopwords)
X['Review Text'] = X['Review Text'].apply(remove_stopwords)

X['Title'] = X['Title'].apply(lambda x: lemm.lemmatize(x))
X['Review Text'] = X['Review Text'].apply(lambda x: lemm.lemmatize(x))

In [ ]:
def remove_punctuation_func(text):
	return re.sub(r'[^a-zA-Z0-9]', ' ', text)

X['Title'] = X['Title'].apply(remove_punctuation_func)
X['Review Text'] = X['Review Text'].apply(remove_punctuation_func)

In [15]:
X['Text'] = list(X['Title']+X['Review Text']+X['Class Name'])
X_train, X_test, y_train, y_test = train_test_split(X['Text'], y, test_size=0.25, random_state=42)

<ipython-input-15-40cd445433d3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Text'] = list(X['Title']+X['Review Text']+X['Class Name'])


In [16]:
#convert the top 10000 most frequent words to tokens, words not in these top words will have their tokens as <OOV>
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

In [18]:
# convert the text in training (X_train) and testing (X_test) datasets into sequences of integers
train_seq = tokenizer.texts_to_sequences(X_train)
test_seq = tokenizer.texts_to_sequences(X_test)

# to pad sequences to a fixed length
train_pad = pad_sequences(train_seq, maxlen=40, truncating="post", padding="post")
test_pad = pad_sequences(test_seq, maxlen=40, truncating="post", padding="post")

In [19]:
# RNN
model = keras.models.Sequential()
model.add(keras.layers.Embedding(10000, 128))
model.add(keras.layers.SimpleRNN(64, return_sequences=True))
model.add(keras.layers.SimpleRNN(64))
model.add(keras.layers.Dense(128, activation="relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, None, 64)          12352     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 64)                8256      
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1309057 (4.99 MB)
Trainable params: 130905

In [20]:
model.compile("rmsprop","binary_crossentropy", metrics=["accuracy"])
history = model.fit(train_pad, y_train, epochs=5)

Epoch 1/5
551/551 [==============================] - 16s 25ms/step - loss: 0.4084 - accuracy: 0.8295
Epoch 2/5
551/551 [==============================] - 11s 20ms/step - loss: 0.2961 - accuracy: 0.8839
Epoch 3/5
551/551 [==============================] - 11s 20ms/step - loss: 0.2607 - accuracy: 0.9010
Epoch 4/5
551/551 [==============================] - 11s 20ms/step - loss: 0.2151 - accuracy: 0.9214
Epoch 5/5
551/551 [==============================] - 11s 20ms/step - loss: 0.1697 - accuracy: 0.9397


In [21]:
test_loss, test_accuracy = model.evaluate(test_pad, y_test)

# Print the test loss and accuracy
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


184/184 [==============================] - 1s 5ms/step - loss: 0.4408 - accuracy: 0.8695
Test Loss: 0.4407849907875061
Test Accuracy: 0.8694614768028259


In [22]:
import numpy as np

single_test_instance = test_pad[9:10]

predicted_probability = model.predict(single_test_instance)[0, 0]

predicted_label = 1 if predicted_probability > 0.5 else 0

# Get the true label for the single instance
true_label = y_test[9:10]


print("Predicted Probability:", predicted_probability)
print("Predicted Label:", predicted_label)
print("True Label:", true_label)


is_correct = np.equal(predicted_label, true_label)
print("Is Correct Prediction?", is_correct)


1/1 [==============================] - 0s 246ms/step
Predicted Probability: 0.14802967
Predicted Label: 0
True Label: 20310    0
Name: Rating, dtype: int64
Is Correct Prediction? 20310    True
Name: Rating, dtype: bool
